In [110]:
from IPython.display import HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# Uploading to SynBioHub

Using this notebook will allow you to visualize your Excel data and store it in SynBioHub. First, make sure your Excel spreadsheet follows the template provided by the TASBE-Flow Analytics Tutorial: https://github.com/TASBE/TASBEFlowAnalytics-Tutorial/blob/master/batch_template.xlsm

After downloading the accompanying python code to this notebook, make sure that it is in the same directory as your Excel file.

In [111]:
from sbol import *
import re
import sys
import xlrd
from ipywidgets import widgets
import getpass

In [112]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Select File:

Select the Excel spreadsheet you want to model.

In [113]:
from IPython.display import display
from ipyupload import FileUpload

w = FileUpload(
    accept='',
    multiple=False,
    disabled=False,
    style_button='color: white; background-color: darkorange; width: 180px;', 
    compress_level=0
)

display(w)

FileUpload(style_button='color: white; background-color: darkorange; width: 180px;')

## Checking File Compatability:
Press the button below to confirm:

In [114]:
##CHECK ALL POSSIBLE ERRORS AND SEE IF THEY HAVE BEEN CAUGHT

from tqdm import tqdm
import time
import os

ActionButton = widgets.Button(
    description = 'Proceed'
)
out = widgets.Output()
def on_button_clicked(_):
    with out:
        clear_output()
    with tqdm(total=1000) as pbar:
        file_location = list(w.value)
        file_location = file_location[0]
        pbar.set_description("Converting \"%s\" to SBOL..." % file_location)
        pbar.update(59)
        time.sleep(0.1)
        name,extension = os.path.splitext(file_location)
        counter = 0
        for i in ['.xlsm','.xlsx','.xltx','.xltm','.xls','.xlm','.xlt']:
            if extension == i:
                counter = 1
                break
        if counter != 1:
            print('Error: "%s" file format is not supported.' % extension)
        else:    
            %run -i 'experimentdnaexcel.py'
            pbar.update(59)
            time.sleep(0.1)

            global doc
            pbar.update(59)
            time.sleep(0.1)         
            doc = Document()
            pbar.update(59)
            time.sleep(0.1)
            setHomespace('http://bu.edu/dasha')       
            pbar.update(59)
            time.sleep(0.1)    
            Config.setOption('sbol_typed_uris',False)
            pbar.update(59)
            time.sleep(0.1)
            Config.setOption('sbol_compliant_uris',True)
            pbar.update(59)
            time.sleep(0.1)

            wb = MakeBook(file_location)
            pbar.update(59)
            time.sleep(0.1)
            (ExpName, ExpSheet) = ExcelImport(wb)
            if (ExpName == -1 and ExpSheet == -1):
                return
            pbar.update(59)
            time.sleep(0.1)
            Unit = UnitCollectionFunc(ExpSheet)
            if Unit == -1:
                return
            pbar.update(59)
            time.sleep(0.1)
            (ModList,PlasmidList_orig) = PlasModList(ExpSheet)
            if (ModList == -1 and PlasmidList_orig == -1):
                return
            pbar.update(59)
            time.sleep(0.1)
            PlasmidList_norepeat = PlasNoRepeat(PlasmidList_orig)
            pbar.update(59)
            time.sleep(0.1)
            NewModList = ModListCleaner(ModList,ExpName)
            pbar.update(59)
            time.sleep(0.1)
            ModDefDict = ModMaker(ExpSheet,ModList,NewModList)
            if ModDefDict == -1:
                return
            pbar.update(59)
            time.sleep(0.1)
            diditwork = SamplesImport(ModList,NewModList,ModDefDict,wb,ExpName)
            if diditwork == -1:
                return
            pbar.update(56)
            time.sleep(0.1)
            CompDefDict = CompMaker(PlasmidList_norepeat)
            pbar.update(59)
            time.sleep(0.1)
            diditwork = FuncMaker(NewModList,ModList,ExpSheet,CompDefDict,ModDefDict,Unit)
            pbar.update(59)
            time.sleep(0.1)

            print('Success!')


ActionButton.on_click(on_button_clicked) 
ActionButton


Button(description='Proceed', style=ButtonStyle())

Converting "20180606_JHT6.xlsm" to SBOL...: 100%|██████████| 1000/1000 [00:01<00:00, 482.84it/s]

Success!


## Enter Collection Details:
Enter the **displayID** (REQUIRED, alphanumeric and underscore characters only, cannot begin with a digit), **collection name** (optional, if you want to elaborate on the displayID), and **collection description**. 

In [115]:
displaywidget = widgets.Text(
    value='',
    description = 'displayID'
)
namewidget = widgets.Text(
    value='',
    description = 'name'
)
descriptionwidget = widgets.Textarea(
    value='',
    description = 'description'
)
collectionbox = widgets.VBox([displaywidget,namewidget,descriptionwidget])
collectionbox


## Enter Login Information:
Enter your **username** and **password** for the SynBioHub account you want to upload your collection to.

In [117]:
usernamewidget = widgets.Text(
    value='',
    placeholder='',
    description='Username:',
    disabled=False
)
uploadButton = widgets.Button(
    description = 'Upload to SynBioHub'
)
passwordwidget = widgets.Password(
    description = 'Password:'
)

out = widgets.Output()
def on_button_clicked(_):
    diditwork = UploadFunc(usernamewidget.value,passwordwidget.value,displaywidget.value,namewidget.value,descriptionwidget.value)

uploadButton.on_click(on_button_clicked)

box = widgets.VBox([usernamewidget,passwordwidget,uploadButton])
box


Successfully uploaded
Login required
